In [ ]:
%env LOG_LEVEL=INFO

In [ ]:
import os
del os.environ["http_proxy"]
del os.environ["https_proxy"]

In [ ]:

import json
import time
from tqdm import tqdm
import numpy as np

In [ ]:
def process_st(dsls):
    new_dsls = []
    # dsls = json.loads(dsls)
    for dsl in dsls:
        input = dsl.get("input", [])
        command = dsl.get("command", "")
        command_args = dsl.get("command_args", {})
        output = dsl.get("output", [])

        input = eval(input)
        output = eval(output)
        command_args = process_dict(command_args)
        for command_arg in command_args:
            command_args[command_arg] = process_dict(command_args[command_arg])
        dsl["input"] = input
        dsl["command"] = command
        dsl["command_args"] = command_args
        dsl["output"] = output
        new_dsls.append(dsl)
    return new_dsls

def process_dict(command_args):
    if isinstance(command_args, dict):
        for command_arg in command_args:
            try:
                if isinstance(eval(command_args[command_arg]), (dict, list, bool)):
                    command_args[command_arg] = eval(command_args[command_arg])
            except Exception as e:
                # print(e)
                # print(command_args)
                pass
    return command_args
  

In [ ]:
%load_ext autoreload
%autoreload 2
from format_template import format_output

In [ ]:
# run using source code 
# import sys
# sys.path.insert(0,"/home/zt/workspace/guidance_vllm")

In [ ]:
import guidance_vllm
from guidance_vllm.llms import VllmServiceModel

In [ ]:
from guidance_vllm.llms._transformers import set_prefix_pos

In [ ]:
model_path = "./models/ckpt-st-231115" # If using svc, this directory only needs to include the tokenizer files

In [ ]:
# if you want to use vllm.LLM loads model
# model = VllmModel(model_path)
# vllm_mode = "local"

In [ ]:
# if you want use the vllm svc
model = VllmServiceModel(model_path)
vllm_mode = "svc"

In [ ]:
# hacking vllm model device to solve errors
model.device = "cpu"

In [ ]:
llm = guidance_vllm.llms.Transformers(model=model, tokenizer=model.tokenizer,vllm_mode=vllm_mode,acceleration=False)#token_healing=False, )

In [ ]:
template = format_output()
input_template = "{{description}}" + template
print(input_template)
character_maker = guidance_vllm(input_template)

In [ ]:
# results lists
result_json = []
result_csv = []

In [ ]:
val_data_path = "data/val_all_st_schema_resample.json"
result_json_path = "data/result_guidance_vllm_prefix_cache.json"
plot_types = ["Area", "Bar", "Line", "Barh", "Box", "Density", "Hist", "Pie", "Scatter"]
with open(val_data_path, "rb") as f:
    # [:100] [100:180] [180:]
    data = json.load(f)
    pbar = tqdm(data)
    # result_json = []
    # result_csv = []
    single_command_rst = {}
    for val_data in pbar:
        llm.cache.clear()
        # guidance.llms.cache.clear()
        rst_line = {}
        inp = val_data["prompt"]
        target = val_data["response"]
        system_prompt = val_data["system"]
        prefix_pos = len(model.tokenizer.encode(system_prompt)) - 1
        set_prefix_pos(prefix_pos)
        # 以最后一个command为准
        target = target.replace("⊕", "").replace("⊗", "")
        target = json.loads(target)
        target = process_st(target)
        command = target[-1]["command"]
        if command not in single_command_rst:
            single_command_rst[command] = {"time": [], "accuracy": [], "numbers": 0}
        # torch.cuda.synchronize()
        start = time.time()
        outputs = character_maker(
            plot_type=plot_types,
            description=system_prompt + "USER: {} ASSISTANT:".format(inp),
            llm=llm,
            silent=True,
            # log=True,
        )

        # torch.cuda.synchronize()
        end = time.time()
        cost_time = round(end - start, 2)
        outputs = str(outputs).split("ASSISTANT:")[-1]
        outputs = outputs.replace("⊕", "").replace("⊗", "")
        try:
            outputs = json.loads(outputs)
            outputs = process_st(outputs)

            if str(target) == str(outputs):
                count = 1
                rst_line["match"] = True
            else:
                count = 0
                rst_line["match"] = False
            rst_line["time"] = cost_time
            rst_line["input"] = inp
            rst_line["target"] = target
            rst_line["pred"] = outputs
            result_json.append(rst_line)
        except Exception as e:
            print(e)
            count = 0
            result_json.append(
                {
                    "output_data_error": {
                        "input": str(inp),
                        "target": str(target),
                        "pred": outputs,
                    }
                }
            )

        single_command_rst[command]["time"].append(cost_time)
        single_command_rst[command]["accuracy"].append(count)

    sum_time = 0
    sun_acc = 0
    sum_num = 0

    for key in single_command_rst:
        assert len(single_command_rst[key]["accuracy"]) == len(
            single_command_rst[key]["time"]
        )
        sum_time += np.sum(single_command_rst[key]["time"])
        sun_acc += np.sum(single_command_rst[key]["accuracy"])
        sum_num += len(single_command_rst[key]["time"])
        single_command_rst[key]["numbers"] = len(single_command_rst[key]["accuracy"])
        single_command_rst[key]["time"] = (
            str(round(np.mean(single_command_rst[key]["time"]), 2)) + "s"
        )
        single_command_rst[key]["accuracy"] = round(
            np.mean(single_command_rst[key]["accuracy"]), 2
        )
    all_mean_time = round(sum_time / sum_num, 2)
    all_mena_acc = round(sun_acc / sum_num, 2)
    print(single_command_rst)

In [ ]:
import pandas as pd
import csv

csv_file_path = "data/result_guidance_vllm_svc_prefix_cache.csv"
with open(csv_file_path, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Command", "Numbers", "single_mean_time", "single_mean_acc", "all_mean_time", "all_mean_acc"])
    for key, value in single_command_rst.items():
        writer.writerow([key, value["numbers"], value["time"], value["accuracy"], all_mean_time, all_mena_acc])
df = pd.read_csv(csv_file_path)
sorted_df = df.sort_values(by="single_mean_acc", ascending=False)
sorted_df.to_csv(csv_file_path, index=False)

with open(result_json_path, 'w', encoding="utf-8") as f3:
    f3.write(json.dumps(result_json, ensure_ascii=False, indent=4))